In [6]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from pymystem3 import Mystem
import pickle

In [5]:
!ls ../data

habrahabr_test.json.zip   sample_submission.csv  train.json
habrahabr_train.json.zip  test.json		 train_target.csv


In [4]:
pd_train = pd.read_json("../data/train.json", lines=True)

In [5]:
Y = pd.read_csv("../data/train_target.csv")

In [57]:
train_title = pd_train['title']

In [117]:
train_content = pd_train['content']

In [92]:
train_data.values[0]

'Самопроизвольное разлогинивание'

In [106]:
title_lem= []

In [116]:
content_lem= []

In [105]:
m = Mystem();
''.join(m.lemmatize(train_data.values[0]))

'самопроизвольный разлогинивание\n'

In [107]:
m = Mystem();
for i in train_data.values:
    title_lem.append(''.join(m.lemmatize(i)))

In [118]:
m = Mystem();
for i in train_content.values:
    content_lem.append(''.join(m.lemmatize(i)))

In [108]:
title_lem[0]

'самопроизвольный разлогинивание\n'

In [121]:
import pickle

In [124]:
pickle.dump(content_lem, open('./content_lem.pickle','wb'))

In [129]:
content_lem_soap = []
for i in tqdm(content_lem):
    content_lem_soap.append(BeautifulSoup(i,'html.parser').text)
    
    

In [132]:
pickle.dump(content_lem_soap, open('./content_lem_soap.pickle','wb'))

In [7]:
content_lem_soap = pickle.load(open('./content_lem_soap.pickle','rb'))

In [8]:
countVect = CountVectorizer(max_features = 10000)
matrix_count_content = countVect.fit_transform(content_lem_soap).toarray()

In [10]:
words = [x[0] for x in sorted(countVect.vocabulary_.items(), key=lambda x: x[1])]
pd.DataFrame(matrix_count_content, columns=words).head(20)

,00,000,0000,000000,00000000,001,00th,01,02,03,...,японский,яркий,ярко,яркость,ярлык,ясно,ясность,ясный,ячейка,ящик
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(matrix_count_content, Y['target'].values, test_size=0.1, random_state=0)

# Modelling

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
reg = LinearRegression(fit_intercept=True)

In [15]:
X_train_raw.shape

(108000, 10000)

In [16]:
Y_train.shape

(108000,)

In [44]:
reg.fit(X_train_raw, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [20]:
Y_prediction = reg.predict(X_test_raw)

# Evaluate

In [42]:
from sklearn.metrics import mean_absolute_error

In [43]:
mean_absolute_error(Y_test, Y_prediction)

1.1242363451733135

# Validation

In [25]:
%%time
pd_test = pd.read_json("../data/test.json", lines=True)

CPU times: user 16.6 s, sys: 3.66 s, total: 20.3 s
Wall time: 20.2 s


In [28]:
test_content = pd_test['content']

In [31]:
test_content_lem = []
m_test = Mystem();
for i in tqdm(test_content.values):
    test_content_lem.append(''.join(m_test.lemmatize(i)))

Widget Javascript not detected.  It may not be installed or enabled properly.


In [32]:
test_content_lem_soap = []
for i in tqdm(test_content_lem):
    test_content_lem_soap.append(BeautifulSoup(i,'html.parser').text)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [35]:
pickle.dump(test_content_lem_soap, open('./test_content_lem_soap.pickle','wb'))

In [36]:
matrix_count_test_content = countVect.transform(test_content_lem_soap).toarray()

In [37]:
%%time
V_result = pd.read_json("../data/test.json", lines=True)[["_id"]].rename(columns={"_id": "url"})

CPU times: user 16.7 s, sys: 7.94 s, total: 24.6 s
Wall time: 24.6 s


In [38]:
V_result["target"] = reg.predict(matrix_count_test_content)

In [39]:
output_name = "../results/content_cv10k_linreg.csv"

In [40]:
V_result.to_csv(output_name, index=False)

In [41]:
!head $output_name

url,target
https://geektimes.ru/post/87455/,1.8160771901190897
https://geektimes.ru/post/87452/,1.5893218036603642
https://geektimes.ru/post/87459/,1.9189423560839824
https://habrahabr.ru/post/87461/,2.0620800496288743
https://habrahabr.ru/post/5754/,1.8688625481678947
https://geektimes.ru/post/87460/,3.9536313364109157
https://geektimes.ru/post/87462/,2.650042096711365
https://habrahabr.ru/post/87467/,2.1466267056844544
https://habrahabr.ru/post/87464/,3.117150439519853


In [23]:
matrix_count = countVect.fit_transform(title_lem).toarray()

NameError: name 'title_lem' is not defined

In [114]:
matrix_count.shape

(120000, 55359)

In [115]:
words = [x[0] for x in sorted(countVect.vocabulary_.items(), key=lambda x: x[1])]
pandas.DataFrame(matrix_count, columns=words).head(20)

,00,000,0000,000000,00000000,000008,0001,000webhost,000р,001,...,ящер,ящерица,ящик,яя,іi,іxia,її,қазақстан,कर,螢火蟲
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
pd_train['title']

0                           Самопроизвольное разлогинивание
1         Stand-along cообщества против сообществ в рамк...
2                                       К нам скунс заходил
3         Cisco подаёт в суд на Apple за использование т...
4         Подробности о Medison Celebrity: халявы не буд...
5         FasterN9 для Nokia N9 (MeeGo Harmattan): улучш...
6         Ubuntu Phone можно будет купить в октябре 2013...
7          Raspberry Pi + Siri = голосовое управление домом
8                 Опрос по Октябрьскому релизу Ubuntu Phone
9         Cascade File System или размышления о файловой...
10        Несоответствие действительности в рекламе ориг...
11        HDR-видео и Mobile Bravia Engine 2 в смартфона...
12                           GPS-мониторинг без лишних глаз
13                         Развёртывание репозиториев Linux
14              Цепь эффективных интернет-продаж: концепция
15               Apple ответит за аккумулятор iPhone в суде
16              Так все-таки Total Comma